In [8]:
from multiprocessing import Process
import os
import numpy as np

def ping(ttl: int, K: int, L: int, target_name: str, result_file = "ping_result"):
#-> np.ndarray:
    """
    This function adds two numbers.
    
    Parameters:
    ttl (int):      time to live
    K (int):        number of packets sent to the target
    L (int):        number of bytes for each packet
    target_name:    target of the ping command
    result_file:    name of the txt file in which the result is store
    
    Returns:
    np.ndarray: one row for each one of the K pings, 
                on the first column the values of time
                on the second column the values of TTL
    """
    result_file = f"{result_file}_{L:04d}.txt"
    os.system(f'ping {target_name} -i {ttl} -n {K} -l {L} > {result_file}')

    f = open(result_file)
    result_string = f.read()
    # print(result_string)

    result_matrix = np.empty([0,2], dtype=int)
    
    for line in result_string.splitlines():
        word_before_time = "durata="
        time_unit = "ms"
        index = line.find(word_before_time)
        if index == -1:
            continue
        time_and_ttl = line[index + len(word_before_time):]
        time = (int) (time_and_ttl[0:time_and_ttl.find(time_unit)])
        ttl = (int) (time_and_ttl[time_and_ttl.find("=")+1:])
        
        result_matrix = np.vstack([result_matrix, [time, ttl]])
    
    # os.remove(result_file)
    return result_matrix



""" for l in range(10, 1472, 100):
    ttl = 30
    K = 10 # number of packets
    L = 20 # number of bytes
    target_name = "atl.speedtest.clouvider.net"

    matrix = ping(ttl, K, l, target_name)
    print(matrix) """

if __name__ == "__main__":
    p1 = Process(target=ping, args=[30,10,10,"atl.speedtest.clouvider.net"])
    p1.start()
    p2 = Process(target=ping, args=[30,10,20,"atl.speedtest.clouvider.net"])
    p2.start()
    p1.join()
    p2.join()
